In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import networkx as nx
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.image as mpimg
import random

In [3]:
pip install openpyxl

     |████████████████████████████████| 249 kB 1.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
#dataset = pd.read_excel('/kaggle/input/vitrina/vitrina_clusters.xlsx')
dataset = pd.read_excel('vitrina_clusters.xlsx')
dataset = dataset[:400]

In [5]:
dataset.head()

,Unnamed: 0,Phone_new,Number of bills,Average bill,Average number of goods in bill,Revenue,Discount,%TN_Автотовары,%TN_Аксессуары,%TN_Детские товары,...,%TN_Мелкая бытовая техника,%TN_Сделай сам,%TN_Спорт и активный отдых,%TN_ТВ-Аудио,%TN_Товары для дома,%TN_Услуги,"%TN_Хобби, досуг",%TN_Цифровая Техника,%TN_Элитная техника,cluster
0,1,32565748-000000000,8,2469.625,1.0,19757,0,0.0,37.5,12.5,...,12.5,0.0,0.0,25.0,0.0,0.0,0,0.0,0,0
1,2,32565750-575448565151 а,1,26599.000,2.0,26599,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,5
2,3,32565750-575448565151 а5,1,1590.000,1.0,1590,0,0.0,0.0,0.0,...,100.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0
3,4,43553240-575054413251+75,2,5064.500,1.0,10129,0,0.0,0.0,0.0,...,50.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0
4,5,55445748-514957495657715,1,27449.000,1.0,27449,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,5


In [6]:
dataset["cluster"].value_counts()

0    337
5     51
3      9
4      3
Name: cluster, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
df1 = dataset[dataset['cluster'] == 0]
#df2 = dataset[dataset['cluster'] == 1]
#df3 = dataset[dataset['cluster'] == 2]
df4 = dataset[dataset['cluster'] == 3]
df5 = dataset[dataset['cluster'] == 4]
df6 = dataset[dataset['cluster'] == 5]

X1_train, X1_test = train_test_split(df1,test_size=0.2, random_state=42)
#X2_train, X2_test = train_test_split(df2,test_size=0.2, random_state=42)
#X3_train, X3_test = train_test_split(df3,test_size=0.2, random_state=42)
X4_train, X4_test = train_test_split(df4,test_size=0.2, random_state=42)
X5_train, X5_test = train_test_split(df5,test_size=0.2, random_state=42)
X6_train, X6_test = train_test_split(df6,test_size=0.2, random_state=42)

In [8]:
def creating_graph(dataset):
    G = nx.Graph()
    node2index = {}
    node_counter = 0
    
    some_unique_id = "000-000"
    G.add_node(node_counter,type='user') #- вершина для тестового набора данных
    node2index[some_unique_id] = node_counter
    node_counter += 1
    
    items = ['%TN_Автотовары', '%TN_Аксессуары', '%TN_Детские товары', '%TN_Игры, софт и развлечения', '%TN_Климат', '%TN_Крупная бытовая техника', '%TN_Мебель', '%TN_Мелкая бытовая техника', '%TN_Сделай сам', '%TN_Спорт и активный отдых', '%TN_ТВ-Аудио', '%TN_Товары для дома', '%TN_Услуги', '%TN_Хобби, досуг', '%TN_Цифровая Техника', '%TN_Элитная техника']
    
    for i in items:
        G.add_node(node_counter, type='item')
        node2index[i] = node_counter
        node_counter += 1
    
    for index, row in dataset.iterrows():
        G.add_node(node_counter, type='user')
        node2index[row['Phone_new']] = node_counter
        for i in items:
            G.add_edge(node_counter, node2index[i], weight=row[i])
        node_counter += 1
    
    return G, node2index
    

In [9]:
def biased_random_walk(G, start_node, walk_length, p=1, q=1):
    walk = [start_node]

    while len(walk) < walk_length:
        cur_node = walk[-1]
        cur_neighbors = list(G.neighbors(cur_node))

        if len(cur_neighbors) > 0:
            if len(walk) == 1:
                walk.append(random.choice(cur_neighbors))
            else:
                prev_node = walk[-2]

                probability = []
                for neighbor in cur_neighbors:
                    if neighbor == prev_node:
                        # Return parameter 
                        probability.append(1/p)
                    elif G.has_edge(neighbor, prev_node):
                        # Stay parameter 
                        probability.append(1)
                    else:
                        # In-out parameter 
                        probability.append(1/q)

                probability = np.array(probability)
                probability = probability / probability.sum()  # normalize

                next_node = np.random.choice(cur_neighbors, p=probability)
                walk.append(next_node)
        else:
            break

    return walk

In [10]:
def generate_walks(G, num_walks, walk_length, p=1, q=1):
    walks = []
    nodes = list(G.nodes())
    for _ in range(num_walks):
        random.shuffle(nodes)  # to ensure randomness
        for node in nodes:
            walk_from_node = biased_random_walk(G, node, walk_length, p, q)
            walks.append(walk_from_node)
    return walks

In [28]:
!pip install pygsp

^C
ERROR: Operation cancelled by user


In [11]:

def creating_model(G):
    walks = generate_walks(G, num_walks=10, walk_length=20, p=9, q=1)
    filtered_walks = [walk for walk in walks if len(walk) >= 5]

    # to String  (for Word2Vec input)
    walks = [[str(node) for node in walk] for walk in walks]

    # Word2Vec train
    model = Word2Vec(walks, vector_size=128, window=5, min_count=0,  hs=1, sg=1, workers=4, epochs=10)

    # node embedding extract
    embeddings = {node_id: model.wv[node_id] for node_id in model.wv.index_to_key}
    
    return model

In [12]:
G1, node2index1 = creating_graph(X1_train)
G4, node2index4 = creating_graph(X4_train)
G5, node2index5 = creating_graph(X5_train)
G6, node2index6 = creating_graph(X6_train)

In [13]:
model1 = creating_model(G1)
model4 = creating_model(G4)
model5 = creating_model(G5)
model6 = creating_model(G6)

In [14]:
embeddings1 = {node_id: model1.wv[node_id] for node_id in model1.wv.index_to_key}
embeddings4 = {node_id: model4.wv[node_id] for node_id in model4.wv.index_to_key}
embeddings5 = {node_id: model5.wv[node_id] for node_id in model5.wv.index_to_key}
embeddings6 = {node_id: model6.wv[node_id] for node_id in model6.wv.index_to_key}

In [49]:
def get_user_embedding(user_id, node2index, embeddings):
    return embeddings[str(node2index[str(user_id)])]

In [50]:
def get_rated_items(user_id, df, item, items):
    return df[df['Phone_new'] == user_id][item]

In [51]:
def calculate_similarities(user_id, df, node2index, embeddings, items):
    for item in items:
        rated_items = get_rated_items(user_id, df, item, items)
    user_embedding = get_user_embedding(user_id, node2index, embeddings)

    item_similarities = []
    for item_id in items:
        if item_id not in rated_items:  
            item_embedding = embeddings[str(node2index[item_id])]
            similarity = cosine_similarity([user_embedding], [item_embedding])[0][0]
            item_similarities.append((item_id, similarity))

    return item_similarities

In [52]:
def recommend_items(user_id, df, node2index, embeddings, items, num_items=5):
    for item in items:
        rated_items = get_rated_items(user_id, df, item, items)
    
    #print(f"User {user_id} has purchased:")
    #print(rated_items)
    
    item_similarities = calculate_similarities(user_id, df, node2index, embeddings, items)

    recommended_items = sorted(item_similarities, key=lambda x: x[1], reverse=True)[:num_items]

    #print(f"\nRecommended items for user {user_id}:")
    #print(recommended_items)
    return recommended_items

In [56]:
items = ['%TN_Автотовары', '%TN_Аксессуары', '%TN_Детские товары', '%TN_Игры, софт и развлечения', '%TN_Климат', '%TN_Крупная бытовая техника', '%TN_Мебель', '%TN_Мелкая бытовая техника', '%TN_Сделай сам', '%TN_Спорт и активный отдых', '%TN_ТВ-Аудио', '%TN_Товары для дома', '%TN_Услуги', '%TN_Хобби, досуг', '%TN_Цифровая Техника', '%TN_Элитная техника']
recommends1 = recommend_items("000-000", df1, node2index1, embeddings1, items, num_items=16)
recommends4 = recommend_items("000-000", df4, node2index4, embeddings4, items, num_items=16)
recommends5 = recommend_items("000-000", df5, node2index5, embeddings5, items, num_items=16)
recommends6 = recommend_items("000-000", df6, node2index6, embeddings6, items, num_items=16)

In [57]:
recommends1

[('%TN_Хобби, досуг', 0.13726526),
 ('%TN_Товары для дома', 0.11103001),
 ('%TN_Игры, софт и развлечения', 0.066858724),
 ('%TN_Элитная техника', 0.062180065),
 ('%TN_Крупная бытовая техника', 0.06067749),
 ('%TN_ТВ-Аудио', 0.058528956),
 ('%TN_Мебель', 0.05688161),
 ('%TN_Цифровая Техника', 0.05383847),
 ('%TN_Детские товары', 0.03802485),
 ('%TN_Спорт и активный отдых', 0.03440445),
 ('%TN_Сделай сам', 0.033086687),
 ('%TN_Мелкая бытовая техника', 0.026368296),
 ('%TN_Услуги', 0.022555865),
 ('%TN_Автотовары', 0.014402358),
 ('%TN_Климат', 0.0068117436),
 ('%TN_Аксессуары', -0.006938448)]

In [58]:
X1_test[items]

,%TN_Автотовары,%TN_Аксессуары,%TN_Детские товары,"%TN_Игры, софт и развлечения",%TN_Климат,%TN_Крупная бытовая техника,%TN_Мебель,%TN_Мелкая бытовая техника,%TN_Сделай сам,%TN_Спорт и активный отдых,%TN_ТВ-Аудио,%TN_Товары для дома,%TN_Услуги,"%TN_Хобби, досуг",%TN_Цифровая Техника,%TN_Элитная техника
59,0.0,60.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,20.0,0.0,0,0.0,0
197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
30,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0,0.0,0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0,0.0,0
390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,100.0,0


In [59]:
import numpy as np
from sklearn.metrics import average_precision_score, ndcg_score

def calculate_map(true_relevance, relevance_score):
    ap = average_precision_score(true_relevance, relevance_score)
    map_metric = np.mean(ap)
    return map_metric

def calculate_ndcg(true_relevance, relevance_score):
    dcg = ndcg_score(true_relevance, relevance_score)
    idcg = ndcg_score(true_relevance, true_relevance)
    ndcg = dcg / idcg
    return ndcg

# Example usage
# for i in items:
#     if dataset[]
# true_relevance = np.asarray(purchased)
# relevance_score = np.asarray(recommends)

# map_metric = calculate_map(true_relevance, relevance_score)
# ndcg = calculate_ndcg(true_relevance, relevance_score)

# print("MAP:", map_metric)
# print("NDCG:", ndcg)

In [60]:
relevance_score1 = [x[1] for x in recommends1]
relevance_score1 = relevance_score1
relevance_score1

[0.13726526,
 0.11103001,
 0.066858724,
 0.062180065,
 0.06067749,
 0.058528956,
 0.05688161,
 0.05383847,
 0.03802485,
 0.03440445,
 0.033086687,
 0.026368296,
 0.022555865,
 0.014402358,
 0.0068117436,
 -0.006938448]

In [61]:
X1_test[items].iloc[0].values.tolist()

[0.0,
 60.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 20.0,
 0.0,
 0.0,
 0.0,
 20.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [62]:
true_relevance = X1_test[items].iloc[0].values.tolist()
true_relevance

[0.0,
 60.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 20.0,
 0.0,
 0.0,
 0.0,
 20.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [63]:
def map_at_k(scores, true_relevance, k):
    """
    Calculates the Mean Average Precision at K (MAP@K) metric for a set of predictions.

    Parameters:
    scores (list of lists): A list of lists, where each inner list contains the predicted scores for a single user.
    true_relevance (list of lists): A list of lists, where each inner list contains the true relevance for a single user.
    k (int): The number of items to consider for each user.

    Returns:
    float: The MAP@K score.
    """
    num_users = len(scores)
    ap_scores = []

    for i in range(num_users):
        # Sort the predicted scores by value
        sorted_scores = sorted(enumerate(scores[i]), key=lambda x: x[1], reverse=True)
        # Get the top k predicted items
        top_k_predicted_items = [x[0] for x in sorted_scores[:k]]
        # Calculate the average precision
        num_relevant_items = sum(1 for j in range(len(true_relevance[i])) if true_relevance[i][j] > 0)
        relevant_count = 0
        ap = 0
        for j in range(k):
            if true_relevance[i][top_k_predicted_items[j]] > 0:
                relevant_count += 1
                ap += relevant_count / (j + 1)
        ap_scores.append(ap / min(num_relevant_items, k))

    # Calculate the MAP@K score
    map = np.mean(ap_scores)
    return map

In [64]:
map_at_k([true_relevance], [relevance_score1], 5)

1.0

In [70]:
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the GCN layer
class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)
        nn.init.zeros_(self.bias)

    def forward(self, input, adj):
        support = torch.matmul(input, self.weight)
        output = torch.matmul(adj, support) + self.bias
        return output

# Define the GCN model
class GCN(nn.Module):
    def __init__(self, n_features, n_hidden, n_classes):
        super(GCN, self).__init__()
        self.gc1 = GCNLayer(n_features, n_hidden)
        self.relu = nn.ReLU()
        self.gc2 = GCNLayer(n_hidden, n_classes)

    def forward(self, x, adj):
        x = self.gc1(x, adj)
        x = self.relu(x)
        x = self.gc2(x, adj)
        return x

# Create a graph using NetworkX
G = G1

# Convert the graph to a PyTorch tensor
# добавить индексы 
edge_index = torch.tensor([[e[0] for e in G.edges()], [e[1] for e in G.edges()]])

# Create the feature matrix
features = torch.randn(X1_train.shape[0], 10) # заглушка
# features = X1_train.

# Convert the feature matrix to a PyTorch tensor
x = torch.tensor(features.tolist())

# Create the adjacency matrix
adj = nx.adjacency_matrix(G).toarray()

# Convert the adjacency matrix to a PyTorch tensor
adj = torch.tensor(adj)

# Create the GCN model
model = GCN(10, 16, 4)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train the model
for epoch in range(100):
    optimizer.zero_grad()
    output = model(x, adj)
    loss = criterion(output, torch.randint(4, (100,)))
    loss.backward()
    optimizer.step()

# Use the trained model to generate embeddings
embeddings = model(x, adj)

ImportError: cannot import name '_functionalize_sync' from 'torch._utils' (/Users/georgij/opt/anaconda3/envs/causalml-py38/lib/python3.8/site-packages/torch/_utils.py)